# Advanced Sign Optimization: Benchmark Comparison

This notebook compares multiple approaches for resolving the nematic sign ambiguity (n ≡ -n) in FCPM director reconstruction.

## Approaches Compared

1. **V2 Layer+Refine** (baseline): Layer propagation followed by iterative refinement
2. **Graph Cuts**: Min-cut/max-flow for globally optimal solution
3. **Simulated Annealing**: Metropolis-Hastings with adaptive temperature
4. **Hierarchical**: Multi-scale coarse-to-fine optimization
5. **Belief Propagation**: Message passing on factor graph

## Metrics

- **Energy**: Gradient energy Σ|n_i - n_j|² (lower is better)
- **Angular Error**: Deviation from ground truth (lower is better)
- **Time**: Computation time (lower is better)

In [ ]:
# Setup
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import time

# Add paths
sys.path.insert(0, str(Path('.').resolve().parent))
sys.path.insert(0, str(Path('.').resolve()))

import fcpm
from fcpm.reconstruction import reconstruct_via_qtensor

# Import approaches
from sign_optimization_v2 import layer_then_refine, compute_gradient_energy
from approaches.graph_cuts import GraphCutsOptimizer
from approaches.simulated_annealing import SimulatedAnnealingOptimizer, SimulatedAnnealingConfig
from approaches.hierarchical import HierarchicalOptimizer
from approaches.belief_propagation import BeliefPropagationOptimizer, BeliefPropagationConfig

print("All imports successful!")
print(f"FCPM version: {fcpm.__version__ if hasattr(fcpm, '__version__') else 'unknown'}")

## 1. Create Test Data

Generate a cholesteric director field and simulate noisy FCPM measurements.

In [ ]:
# Test parameters
SHAPE = (48, 48, 24)
PITCH = 6.0
NOISE_LEVEL = 0.05  # 5% noise

# Create ground truth
print("Creating ground truth director...")
director_gt = fcpm.create_cholesteric_director(shape=SHAPE, pitch=PITCH)
print(f"Shape: {director_gt.shape}")

# Simulate FCPM
print("\nSimulating FCPM...")
I_fcpm = fcpm.simulate_fcpm(director_gt)
print(f"FCPM shape: {I_fcpm.shape}")

# Add noise
print(f"\nAdding {NOISE_LEVEL*100:.0f}% noise...")
I_fcpm_noisy = fcpm.add_fcpm_realistic_noise(
    I_fcpm, noise_model='mixed', gaussian_sigma=NOISE_LEVEL
)
I_fcpm_noisy = fcpm.normalize_fcpm(I_fcpm_noisy)

# Reconstruct (without sign fix)
print("\nReconstructing via Q-tensor (no sign fix)...")
director_raw, Q, info = reconstruct_via_qtensor(I_fcpm_noisy)
print(f"Reconstruction complete.")

# Initial metrics
raw_energy = compute_gradient_energy(director_raw.to_array())
raw_metrics = fcpm.summary_metrics(director_raw, director_gt)
print(f"\nRaw (no sign fix):")
print(f"  Energy: {raw_energy:.2f}")
print(f"  Angular error: {raw_metrics['angular_error_mean_deg']:.2f}° (mean)")

## 2. Run All Approaches

Compare each optimization approach on the same test data.

In [ ]:
results = {}

# V2 Layer+Refine (baseline)
print("="*60)
print("V2 Layer+Refine (baseline)")
print("="*60)
t0 = time.time()
result_v2 = layer_then_refine(director_raw, verbose=True)
time_v2 = time.time() - t0
metrics_v2 = fcpm.summary_metrics(result_v2.director, director_gt)
results['V2 Layer+Refine'] = {
    'result': result_v2,
    'metrics': metrics_v2,
    'time': time_v2
}
print(f"\nTime: {time_v2:.2f}s")
print(f"Angular error: {metrics_v2['angular_error_mean_deg']:.2f}°")

In [ ]:
# Graph Cuts
print("="*60)
print("Graph Cuts")
print("="*60)
optimizer_gc = GraphCutsOptimizer()
t0 = time.time()
result_gc = optimizer_gc.optimize(director_raw, verbose=True)
time_gc = time.time() - t0
metrics_gc = fcpm.summary_metrics(result_gc.director, director_gt)
results['Graph Cuts'] = {
    'result': result_gc,
    'metrics': metrics_gc,
    'time': time_gc
}
print(f"\nTime: {time_gc:.2f}s")
print(f"Angular error: {metrics_gc['angular_error_mean_deg']:.2f}°")

In [ ]:
# Simulated Annealing
print("="*60)
print("Simulated Annealing")
print("="*60)
config_sa = SimulatedAnnealingConfig(
    max_iterations=30000,
    use_cluster_moves=True,
    use_adaptive=True
)
optimizer_sa = SimulatedAnnealingOptimizer(config_sa)
t0 = time.time()
result_sa = optimizer_sa.optimize(director_raw, verbose=True)
time_sa = time.time() - t0
metrics_sa = fcpm.summary_metrics(result_sa.director, director_gt)
results['Simulated Annealing'] = {
    'result': result_sa,
    'metrics': metrics_sa,
    'time': time_sa
}
print(f"\nTime: {time_sa:.2f}s")
print(f"Angular error: {metrics_sa['angular_error_mean_deg']:.2f}°")

In [ ]:
# Hierarchical
print("="*60)
print("Hierarchical Coarse-to-Fine")
print("="*60)
optimizer_hier = HierarchicalOptimizer()
t0 = time.time()
result_hier = optimizer_hier.optimize(director_raw, verbose=True)
time_hier = time.time() - t0
metrics_hier = fcpm.summary_metrics(result_hier.director, director_gt)
results['Hierarchical'] = {
    'result': result_hier,
    'metrics': metrics_hier,
    'time': time_hier
}
print(f"\nTime: {time_hier:.2f}s")
print(f"Angular error: {metrics_hier['angular_error_mean_deg']:.2f}°")

In [ ]:
# Belief Propagation
print("="*60)
print("Belief Propagation")
print("="*60)
config_bp = BeliefPropagationConfig(
    max_iterations=100,
    damping=0.5,
    beta=2.0
)
optimizer_bp = BeliefPropagationOptimizer(config_bp)
t0 = time.time()
result_bp = optimizer_bp.optimize(director_raw, verbose=True)
time_bp = time.time() - t0
metrics_bp = fcpm.summary_metrics(result_bp.director, director_gt)
results['Belief Propagation'] = {
    'result': result_bp,
    'metrics': metrics_bp,
    'time': time_bp
}
print(f"\nTime: {time_bp:.2f}s")
print(f"Angular error: {metrics_bp['angular_error_mean_deg']:.2f}°")

## 3. Results Summary

In [ ]:
# Summary table
print("="*80)
print("RESULTS SUMMARY")
print("="*80)
print(f"{'Approach':<25} {'Energy':>12} {'Ang.Err(°)':>12} {'Time(s)':>10}")
print("-"*80)

# Raw baseline
print(f"{'Raw (no fix)':<25} {raw_energy:>12.2f} {raw_metrics['angular_error_mean_deg']:>12.2f} {'N/A':>10}")
print("-"*80)

# All approaches
for name, data in results.items():
    energy = data['result'].final_energy
    error = data['metrics']['angular_error_mean_deg']
    time_s = data['time']
    print(f"{name:<25} {energy:>12.2f} {error:>12.2f} {time_s:>10.2f}")

print("="*80)

# Find best
best_energy = min(results.items(), key=lambda x: x[1]['result'].final_energy)
best_error = min(results.items(), key=lambda x: x[1]['metrics']['angular_error_mean_deg'])
best_time = min(results.items(), key=lambda x: x[1]['time'])

print(f"\nBest energy: {best_energy[0]}")
print(f"Best accuracy: {best_error[0]}")
print(f"Fastest: {best_time[0]}")

## 4. Visual Comparison

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

names = list(results.keys())
colors = plt.cm.tab10(np.linspace(0, 1, len(names)))

# Energy
ax = axes[0]
energies = [results[n]['result'].final_energy for n in names]
bars = ax.bar(range(len(names)), energies, color=colors)
ax.axhline(y=raw_energy, color='red', linestyle='--', label='Raw')
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_ylabel('Final Energy')
ax.set_title('Energy (lower is better)')
ax.legend()

# Angular Error
ax = axes[1]
errors = [results[n]['metrics']['angular_error_mean_deg'] for n in names]
bars = ax.bar(range(len(names)), errors, color=colors)
ax.axhline(y=raw_metrics['angular_error_mean_deg'], color='red', linestyle='--', label='Raw')
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_ylabel('Mean Angular Error (°)')
ax.set_title('Accuracy (lower is better)')
ax.legend()

# Time
ax = axes[2]
times = [results[n]['time'] for n in names]
bars = ax.bar(range(len(names)), times, color=colors)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_ylabel('Time (seconds)')
ax.set_title('Speed (lower is better)')
ax.set_yscale('log')  # Log scale for time

plt.tight_layout()
plt.savefig('comparison_bars.png', dpi=150)
plt.show()

## 5. Director Slice Visualization

In [ ]:
# Visualize director slices
z_mid = SHAPE[2] // 2

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Ground truth
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=axes[0, 0], title='Ground Truth')

# Raw
fcpm.plot_director_slice(director_raw, z_idx=z_mid, step=2, ax=axes[0, 1], title='Raw (no fix)')

# V2
fcpm.plot_director_slice(results['V2 Layer+Refine']['result'].director, z_idx=z_mid, step=2, 
                         ax=axes[0, 2], title='V2 Layer+Refine')

# Graph Cuts
fcpm.plot_director_slice(results['Graph Cuts']['result'].director, z_idx=z_mid, step=2,
                         ax=axes[1, 0], title='Graph Cuts')

# Hierarchical
fcpm.plot_director_slice(results['Hierarchical']['result'].director, z_idx=z_mid, step=2,
                         ax=axes[1, 1], title='Hierarchical')

# Belief Propagation
fcpm.plot_director_slice(results['Belief Propagation']['result'].director, z_idx=z_mid, step=2,
                         ax=axes[1, 2], title='Belief Propagation')

plt.suptitle(f'Director Slices at z={z_mid} (Noise={NOISE_LEVEL*100:.0f}%)', fontsize=14)
plt.tight_layout()
plt.savefig('director_comparison.png', dpi=150)
plt.show()

## 6. Error Maps

In [ ]:
from fcpm.utils.metrics import angular_error_nematic

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

approaches_to_plot = ['V2 Layer+Refine', 'Graph Cuts', 'Simulated Annealing', 
                      'Hierarchical', 'Belief Propagation']

# Raw error
err_raw = angular_error_nematic(director_raw, director_gt) * 180 / np.pi
im = axes[0, 0].imshow(err_raw[:, :, z_mid], cmap='hot', vmin=0, vmax=45)
axes[0, 0].set_title(f'Raw\n(mean={np.mean(err_raw):.1f}°)')
axes[0, 0].axis('off')
plt.colorbar(im, ax=axes[0, 0], fraction=0.046)

# Other approaches
plot_positions = [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
for (row, col), name in zip(plot_positions, approaches_to_plot):
    err = angular_error_nematic(results[name]['result'].director, director_gt) * 180 / np.pi
    im = axes[row, col].imshow(err[:, :, z_mid], cmap='hot', vmin=0, vmax=45)
    axes[row, col].set_title(f'{name}\n(mean={np.mean(err):.1f}°)')
    axes[row, col].axis('off')
    plt.colorbar(im, ax=axes[row, col], fraction=0.046)

plt.suptitle(f'Angular Error Maps at z={z_mid}', fontsize=14)
plt.tight_layout()
plt.savefig('error_maps.png', dpi=150)
plt.show()

## 7. Noise Sensitivity Analysis

Test how each approach performs across different noise levels.

In [ ]:
# This cell may take a while to run!

noise_levels = [0.01, 0.03, 0.05, 0.08, 0.10, 0.15]
noise_results = {name: {'errors': [], 'energies': [], 'times': []} for name in results.keys()}

print("Running noise sensitivity analysis...")
print("This may take several minutes.\n")

for noise in noise_levels:
    print(f"Noise level: {noise*100:.0f}%")
    
    # Generate noisy data
    I_noisy = fcpm.add_fcpm_realistic_noise(
        I_fcpm, noise_model='mixed', gaussian_sigma=noise
    )
    I_noisy = fcpm.normalize_fcpm(I_noisy)
    director_raw_n, _, _ = reconstruct_via_qtensor(I_noisy)
    
    # Test each approach
    for name in results.keys():
        print(f"  {name}...", end=" ", flush=True)
        
        if name == 'V2 Layer+Refine':
            t0 = time.time()
            res = layer_then_refine(director_raw_n, verbose=False)
            t = time.time() - t0
        elif name == 'Graph Cuts':
            t0 = time.time()
            res = GraphCutsOptimizer().optimize(director_raw_n, verbose=False)
            t = time.time() - t0
        elif name == 'Simulated Annealing':
            config = SimulatedAnnealingConfig(max_iterations=20000, use_cluster_moves=True)
            t0 = time.time()
            res = SimulatedAnnealingOptimizer(config).optimize(director_raw_n, verbose=False)
            t = time.time() - t0
        elif name == 'Hierarchical':
            t0 = time.time()
            res = HierarchicalOptimizer().optimize(director_raw_n, verbose=False)
            t = time.time() - t0
        elif name == 'Belief Propagation':
            config = BeliefPropagationConfig(max_iterations=50)
            t0 = time.time()
            res = BeliefPropagationOptimizer(config).optimize(director_raw_n, verbose=False)
            t = time.time() - t0
        
        metrics = fcpm.summary_metrics(res.director, director_gt)
        noise_results[name]['errors'].append(metrics['angular_error_mean_deg'])
        noise_results[name]['energies'].append(res.final_energy)
        noise_results[name]['times'].append(t)
        print(f"{metrics['angular_error_mean_deg']:.1f}°")
    
    print()

print("Done!")

In [ ]:
# Plot noise sensitivity
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
colors = plt.cm.tab10(np.linspace(0, 1, len(noise_results)))

noise_pct = [n * 100 for n in noise_levels]

# Angular error vs noise
ax = axes[0]
for (name, data), color in zip(noise_results.items(), colors):
    ax.plot(noise_pct, data['errors'], '-o', label=name, color=color, linewidth=2, markersize=6)
ax.set_xlabel('Noise Level (%)', fontsize=11)
ax.set_ylabel('Mean Angular Error (°)', fontsize=11)
ax.set_title('Angular Error vs Noise Level', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Energy vs noise
ax = axes[1]
for (name, data), color in zip(noise_results.items(), colors):
    ax.plot(noise_pct, data['energies'], '-o', label=name, color=color, linewidth=2, markersize=6)
ax.set_xlabel('Noise Level (%)', fontsize=11)
ax.set_ylabel('Final Energy', fontsize=11)
ax.set_title('Energy vs Noise Level', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('noise_sensitivity.png', dpi=150)
plt.show()

## 8. Conclusions

### Summary of Approaches

| Approach | Strengths | Weaknesses | Best Use Case |
|----------|-----------|------------|---------------|
| **Graph Cuts** | Globally optimal, fast | Requires submodularity | Smooth fields |
| **Simulated Annealing** | Can escape local minima | Slow, stochastic | High-noise/defects |
| **Hierarchical** | Very fast, global structure | May miss small features | Large volumes |
| **Belief Propagation** | Parallelizable | May not converge | Moderate complexity |
| **V2 Layer+Refine** | Simple, reliable | May miss global structure | General use |

### Recommendations

1. **For most cases**: Use **Graph Cuts** if available, otherwise **V2 Layer+Refine**
2. **For large volumes**: Use **Hierarchical** for speed
3. **For high noise**: Consider **Simulated Annealing** with cluster moves
4. **For validation**: Compare multiple methods

In [ ]:
print("Benchmark complete!")
print("\nSaved files:")
print("  - comparison_bars.png")
print("  - director_comparison.png")
print("  - error_maps.png")
print("  - noise_sensitivity.png")